In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import sys

# 1. Simple RNN Netowork

In [2]:
n_inputs = 3
n_neurons = 5

In [3]:
x0= tf.placeholder(tf.float32, [None, n_inputs])
x1 = tf.placeholder(tf.float32, [None, n_inputs])

In [5]:
wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

y0 = tf.tanh(tf.matmul(x0, wx) + b)
y1 = tf.tanh(tf.matmul(y0, wy) + tf.matmul(x1, wx) + b)

init = tf.global_variables_initializer()

In [6]:
# Mini-batch:        instance 0,instance 1,instance 2,instance 3
x0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t = 0
x1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t = 1

with tf.Session() as sess:
    init.run()
    y0_val, y1_val = sess.run([y0, y1], feed_dict={x0: x0_batch, x1: x1_batch})

In [7]:
print(y0_val)
print(y1_val)
y0_val.shape

[[-0.93580914 -0.87305528  0.99955434 -0.81581593 -0.93684173]
 [-0.96607804 -0.99999994  1.         -0.99948537 -1.        ]
 [-0.98220491 -1.          1.         -0.99999875 -1.        ]
 [ 0.78392994 -1.         -0.99699605 -0.99999946 -1.        ]]
[[ 0.93722236 -1.          1.         -1.         -1.        ]
 [ 0.74702108  0.65820336 -0.44420493 -0.87313861 -0.99701184]
 [ 0.96467161 -1.          0.99999952 -0.99999183 -1.        ]
 [ 0.99798864 -0.99999952  0.99927539  0.81625563 -0.99995649]]


(4, 5)

In [8]:
# with tensorflow API
with tf.variable_scope("basic_rnn"):
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
    output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [x0, x1],
                                                    dtype=tf.float32)
    y0, y1 = output_seqs

In [9]:
with tf.Session() as sess:
    x = tf.constant([[1, 2, 6], [3, 4, 5]])
    y = tf.transpose(x, perm=[1, 0])
    print sess.run(y)
    print sess.run(tf.unstack(x))

[[1 3]
 [2 4]
 [6 5]]
[array([1, 2, 6], dtype=int32), array([3, 4, 5], dtype=int32)]


In [10]:
with tf.variable_scope("basic_rnn_with_transpose", reuse=tf.AUTO_REUSE):
    n_steps = 2
    X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
    X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
    output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs,
                                                    dtype=tf.float32)
    outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

In [11]:
with tf.Session() as sess:
    with tf.variable_scope("basic_rnn_with_transpose"):
        X_batch = np.array([
             # t = 0     t = 1
            [[0, 1, 2], [9, 8, 7]], # instance 0
            [[3, 4, 5], [0, 0, 0]], # instance 1
            [[6, 7, 8], [6, 5, 4]], # instance 2
            [[9, 0, 1], [3, 2, 1]], # instance 3
        ])
        init = tf.global_variables_initializer()
        init.run()
        outputs_val = outputs.eval(feed_dict={X: X_batch})
        print(outputs_val)
    

[[[ 0.7700749   0.92492568  0.55726224 -0.12563048  0.88938397]
  [ 1.          0.99857748  0.99993598  0.98880398  0.99998587]]

 [[ 0.99976492  0.9967165   0.98538113  0.65638256  0.99938405]
  [ 0.80900317  0.16828156  0.46891025  0.12967369 -0.42948887]]

 [[ 0.99999982  0.9998613   0.99961859  0.93530399  0.99999684]
  [ 0.99999839  0.92492437  0.99839371  0.9787792   0.99813163]]

 [[ 0.99771386 -0.99951416  0.81172067  0.99875546 -0.7823742 ]
  [ 0.94335043 -0.69235611  0.65002167  0.90329885  0.723634  ]]]


# Dynamic RNN

In [16]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

# 2. RNN for MNIST

In [17]:
with tf.variable_scope("rnn_mnist"):
    n_steps = 28
    n_inputs = 28
    n_neurons = 150
    n_outputs = 10

    learning_rate = 0.001

    X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
    y = tf.placeholder(tf.int32, [None])

    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
    outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

    logits = tf.layers.dense(states, n_outputs)
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    init = tf.global_variables_initializer()

In [35]:
mnist = input_data.read_data_sets("/tmp/tensorflow")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels
y_test

Extracting /tmp/tensorflow/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/t10k-labels-idx1-ubyte.gz


array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [36]:
with tf.variable_scope("rnn_mnist"):
    n_epochs = 100
    batch_size = 150
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs):
            for iteration in range(mnist.train.num_examples // batch_size):
                X_batch, y_batch = mnist.train.next_batch(batch_size)
                X_batch = X_batch.reshape((-1, n_steps, n_inputs))
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            acc_train = sess.run(accuracy, feed_dict={X: X_batch, y: y_batch})
            acc_test = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
            print("acc_train:", acc_train, "acc_test:", acc_test)

('acc_train:', 0.84666669, 'acc_test:', 0.91100001)
('acc_train:', 0.97333336, 'acc_test:', 0.94800001)
('acc_train:', 0.95333332, 'acc_test:', 0.94950002)
('acc_train:', 0.95333332, 'acc_test:', 0.95880002)
('acc_train:', 0.97333336, 'acc_test:', 0.9637)
('acc_train:', 0.97333336, 'acc_test:', 0.96569997)
('acc_train:', 0.95333332, 'acc_test:', 0.96509999)
('acc_train:', 0.96666664, 'acc_test:', 0.9716)
('acc_train:', 1.0, 'acc_test:', 0.96710002)
('acc_train:', 0.98000002, 'acc_test:', 0.9738)
('acc_train:', 0.98666668, 'acc_test:', 0.97689998)
('acc_train:', 0.98000002, 'acc_test:', 0.9734)
('acc_train:', 0.99333334, 'acc_test:', 0.97659999)
('acc_train:', 0.98000002, 'acc_test:', 0.97439998)
('acc_train:', 1.0, 'acc_test:', 0.97310001)
('acc_train:', 0.98666668, 'acc_test:', 0.977)
('acc_train:', 1.0, 'acc_test:', 0.97359997)
('acc_train:', 0.98666668, 'acc_test:', 0.97320002)
('acc_train:', 0.99333334, 'acc_test:', 0.97409999)
('acc_train:', 0.98000002, 'acc_test:', 0.97589999)
('

# RNN for predict Time Series

In [44]:
with tf.variable_scope("time_series", reuse=tf.AUTO_REUSE):
    n_steps = 20
    n_inputs = 1
    n_neurons = 100
    n_outputs = 1
    X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
    y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
    cell = tf.contrib.rnn.OutputProjectionWrapper(
        tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
        output_size = n_outputs
    )
    outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    # define cost function
    learning_rate = 0.001
    loss = tf.reduce_mean(tf.square(outputs - y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    init = tf.global_variables_initializer()
    # define execution
    n_iterations = 100
    batch_size = 50
    with tf.Session() as sess:
        init.run()
#         for iteration in range(n_iterations):
#             X_batch, y_batch = [...]  # fetch the next training batch
#             sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
#             if iteration % 100 == 0:
#                 mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
#                 print(iteration, "\tMSE:", mse)
    

# Deep RNN

In [49]:
with tf.variable_scope("deep_rnn"):
    n_neurons = 100
    n_layers = 3
    layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                      activation=tf.nn.relu) for layer in range(n_layers)]
    multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
    outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

# With Dropout

In [54]:
with tf.variable_scope("with_dropout", reuse=tf.AUTO_REUSE):
    X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
    y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
    cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
             for layer in range(n_layers)]
    keep_prob = 0.1
    cells = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob)
             for cell in cells]
    multi_layer_cell = tf.contrib.rnn.MultiRNNCell(cells)
    rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

# LSTM

In [3]:
n_neurons = 50
lstm_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [4]:
lstm_cell = tf.contrib.rnn.LSTMCell(num_units=n_neurons, use_peepholes=True)

# GPU Cell

In [7]:
# gpu_cell = tf.contrib.rnn.GPUCell(num_units = n_neurons)

# Nutural Language Processing

In [12]:
with tf.variable_scope("word_embedding"):
    vocabulary_size = 50000
    embedding_size = 150
    init_embeds = tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0)
    embeddings = tf.Variable(init_embeds)
    train_inputs = tf.placeholder(tf.int32, shape=None)
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        value = [0, 1, 2, 3, 4, 5]
        v = sess.run(embed, feed_dict={embed: value})
        print v

[ 0.  1.  2.  3.  4.  5.]
